In [1]:
import os 
import sys
import pygwalker as pyg
import pandas as pd

In [8]:

#Importing Configs
# Define the path where config.py is located
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
Model_Folder = config.Sentiment_models
Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [9]:
df = pd.read_csv(f"{Model_Folder}/{Model}_advanced_sentiment_texts.csv") 
df.fillna(0, inplace=True)
print(df.head())

   Unnamed: 0        date             type  \
0           0  2023-08-31  Isabel Schnabel   
1           1  2023-08-31  Isabel Schnabel   
2           2  2023-08-31  Isabel Schnabel   
3           3  2023-08-31  Isabel Schnabel   
4           4  2023-08-31  Isabel Schnabel   

                                             segment sentiment  confidence  
0  speech disinflation and the phillips curve spe...       pos         1.0  
1  activity has moderated visibly , and forwardlo...       neg         0.8  
2  but important pockets of resilience remain , e...       neg         0.6  
3  headline inflation has come down , mainly on t...       neg         0.8  
4  but underlying price pressures remain stubborn...       pos         0.6  


In [12]:
pd.get_dummies(df['sentiment'], dtype=float)

,neg,pos
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0
...,...,...
243393,1.0,0.0
243394,1.0,0.0
243395,1.0,0.0
243396,0.0,1.0


In [18]:
x = 1 
for i in range(90000000):
    x = x*2
    x=x/10

In [19]:
def test(number_man):
    number_man= number_man*2
    number_man = number_man/10
    return(number_man)

y = 1

for i in range(90000000):
    y = test(y)


In [25]:
df['ym'] = df['date'].apply(lambda x: x[:7])


#df2 = df.groupby('ym', as_index = False).sum()

#df.drop(columns=['Unnamed: 0', 'date', 'type', 'segment'], inplace=True)
#df2 = df.groupby('ym', as_index = False).mean()

df.drop(columns=['Unnamed: 0', 'ym', 'type', 'segment'], inplace=True)
df2 = df.groupby('date', as_index = False).mean()


In [26]:
print(df2.head())

         date      good       bad   neutral  compound
0  2007-01-15  0.119555  0.016580  0.863868  0.311062
1  2007-01-16  0.004070  0.177351  0.818587 -0.479030
2  2007-01-18  0.082542  0.045634  0.871844  0.136354
3  2007-01-19  0.093083  0.029201  0.877681  0.219633
4  2007-01-25  0.113418  0.042932  0.843620  0.227966


In [27]:
gwalker = pyg.walk(df2)

Box(children=(HTML(value='<div id="ifr-pyg-2" style="height: auto">\n    <head>\n        <meta http-equiv="Con…